In [1]:
from collections import defaultdict

Id2line = {}
Type2Id = defaultdict(list)

In [2]:
with open("Posts.xml", "r") as f:
    f.readline()
    f.readline()
    for line in f.readlines():
        line = line.strip()   
        if line == "</posts>":
            continue
        # assert one post
        assert line.startswith("<")
        assert line.endswith("/>"), line
        
        line = line[1:-2]
        post_id = None
        for item in line.split('" ')[:-1]:
            name, value = item.split('="')
            if name == "row Id":
                value = int(value)
                Id2line[value] = line
                post_id = value
            elif name == "PostTypeId":
                value = int(value)
                Type2Id[value].append(post_id)

In [3]:
Type2Id.keys()

dict_keys([1, 2, 5, 4, 7, 6])

In [4]:
hedge2node = []
hedge2nodelabel = []
hedge2nodescore = []

hedge2index = {}
node2index = {}

checkId = {}

In [5]:
for id in Type2Id[1]:
    checkId[id] = True
    line = Id2line[id]
    hindex = len(hedge2index)
    hedge2index[id] = hindex
    hedge2node.append([])
    hedge2nodelabel.append([])
    hedge2nodescore.append([])
    
    qid, qscore = None, None
    for item in line.split('" ')[:-1]:
        name, value = item.split('="')
        if name == "AcceptedAnswerId":
            nid = int(value)
            assert nid not in checkId, nid
            checkId[nid] = True
            nline = Id2line[nid]
            userid, score = None, None
            for item in nline.split('" ')[:-1]:
                name, value = item.split('="')
                if name == "ParentId":
                    assert id == int(value)
                elif name == "OwnerUserId":
                    userid =  int(value)
                    if userid not in node2index:
                        node2index[userid] = len(node2index)
                elif name == "Score":
                    score = int(value)
            if userid != None:
                user_reindex = node2index[userid]
                hedge2node[hindex].append(user_reindex)
                hedge2nodelabel[hindex].append(2)
                hedge2nodescore[hindex].append(score)
        elif name == "OwnerUserId":
            qid = int(value)
            if qid not in node2index:
                node2index[qid] = len(node2index)
        elif name == "Score":
            qscore = int(value)
    if qid != None:
        q_reindex = node2index[qid]
        hedge2node[hindex].append(q_reindex)
        hedge2nodelabel[hindex].append(0)
        hedge2nodescore[hindex].append(qscore)

In [6]:
for id in Type2Id[2]:
    if id in checkId:
        continue
    line = Id2line[id]
    checkId[id] = True
    parentid, userid, score = None, None, None
    for item in line.split('" ')[:-1]:
        name, value = item.split('="')
        if name == "ParentId":
            parentid = int(value)
        elif name == "OwnerUserId":
            userid =  int(value)
            if userid not in node2index:
                node2index[userid] = len(node2index)
        elif name == "Score":
            score = int(value)
    
    if userid != None:
        hindex = hedge2index[parentid]
        user_reindex = node2index[userid]

        hedge2node[hindex].append(user_reindex)
        hedge2nodelabel[hindex].append(1)
        hedge2nodescore[hindex].append(score)

In [7]:
print(len(hedge2node))

26918


In [8]:
with open("hypergraph.txt", "w") as f:
    for hedge in hedge2node:
        if len(hedge) == 0:
            continue
        line = [str(v) for v in hedge]
        f.write("\t".join(line) + "\n")
    
with open("hypergraph_pos.txt", "w") as f:
    for hedge in hedge2nodelabel:
        if len(hedge) == 0:
            continue
        line = [str(lab) for lab in hedge]
        f.write("\t".join(line) + "\n")
    
with open("hypergraph_score.txt", "w") as f:
    for hedge in hedge2nodelabel:
        if len(hedge) == 0:
            continue
        line = [str(sc) for sc in hedge]
        f.write("\t".join(line) + "\n")